# Gramática Libre de Contexto
## Lectura de archivos

In [98]:
file = open(r"C:\Users\hrosa\OneDrive\Documentos\CIC IPN\Primer Semestre\Teoria Computacion\GLC\GLC.txt")

In [99]:
content = file.read()

In [100]:
content.split('\n')

['S -> aC | SB', 'A -> bSCa | ad', 'B -> aSB | bBC', 'C -> aBC | ad', '']

In [101]:
split_content = content.split('\n')
stack = []
for r in split_content:
    rules = []
    for s in r.split():
        rules.append([s])
    stack.append(rules)
print(f'stack: {stack}')

stack: [[['S'], ['->'], ['aC'], ['|'], ['SB']], [['A'], ['->'], ['bSCa'], ['|'], ['ad']], [['B'], ['->'], ['aSB'], ['|'], ['bBC']], [['C'], ['->'], ['aBC'], ['|'], ['ad']], []]


In [103]:
#se crean los diccionarios de cada regla de inferencia
rules = {}
for i in stack:
    if not i:
        continue
    head = i[0][0]
    productions = []
    for j in i[2:]:
        if j[0] != '|':
            productions.append(j[0])
            print(productions)
    rules[head] = productions
print(f'rules{rules}')

['aC']
['aC', 'SB']
['bSCa']
['bSCa', 'ad']
['aSB']
['aSB', 'bBC']
['aBC']
['aBC', 'ad']
rules{'S': ['aC', 'SB'], 'A': ['bSCa', 'ad'], 'B': ['aSB', 'bBC'], 'C': ['aBC', 'ad']}


## Simplificación de unificación no recursiva

In [493]:
def unificacion_no_recursiva(rules):
    #simplificacion de unificacion no recursiva
    for k,v in rules.items():
        # print(f'k:{k}')
        # print(f'v:{v}')
        first = {}
        for i in v: #por cada elemento de producciones
            max_prefix = i #como no se tiene compun, se toma todo el elemento
            # print(f'HOLA I{i}')
            for j in v: #por cada aparición de elemento: comparar contra los demás
                if j != i: #si esa comparación no es igual
                    # print(f'j:{j} != i:{i}')
                    prefix_common = "" #se inicializa en cero, porque aún no hay prefijo comúb
                    compare = 1 #caracter en común en comparación 
                    #mientras no tenga longitud mayor y haya comun
                    #compare 1: a == a
                    #compare 2: ab != B
                    while compare <= len(i) and compare <= len(j) and i[:compare] == j[:compare]:
                        prefix_common = i[:compare]
                        compare += 1

                    counter = 0
                    for production in v: #cuenta # de elementos con prefijo compun
                        if production.startswith(prefix_common):
                            counter += 1
                    if prefix_common and counter > 1: #actualizar el término más común
                        max_prefix = prefix_common

            prefix_found = max_prefix
            body = i[len(prefix_found):] #no incluir el prefijo común

            if prefix_found not in first:
                first[prefix_found] = []
            first[prefix_found].append(body)
    #     print(first)

    # print(f'new rules: {first}')
    new_rules = {}
    counter_r = 1
    # print(rules)
    origin_rules = None
    for k, v in rules.items():
        origin_rules = k

    productions_origin = []

    for k,v in first.items():
        if len(v) == 1: #si solo hay una producción:
            new_rules[k] = v
            productions_origin.append(k+v[0])
        else:
            aux = f"R{counter_r}"
            counter_r += 1
            new_rules[aux] = v
            productions_origin.append(k+aux)

    rules_unified = {origin_rules:productions_origin}
    for k,v in new_rules.items():
        rules_unified[k] = v

    # print(f'new rules: {rules_unified}')

    for k,v in rules_unified.items():
        print(f'{k}->{"|".join(v)}')

    return rules_unified

In [495]:
print(rules)
unificacion_no_recursiva(rules)

{'S': ['bA', 'aB'], 'A': ['bAA', 'aS', 'a'], 'B': ['aBB', 'bS', 'a']}
B->aR1|bS
R1->BB|
bS->


{'B': ['aR1', 'bS'], 'R1': ['BB', ''], 'bS': ['']}

## Simplicidad de símbolos redundantes
1. variables no productivas.
2. variables inalcanzables.

In [104]:
def simbolos_redundantes(rules):
    not_productive = []
    for k,v in rules.items():
        # print(f'k:{k} and v:{v}')
        productive = False

        for production in v:
            terminal_count = 0
            no_terminal_count = False

            for char in production:
                if char.islower(): #es terminal?
                    terminal_count += 1
                if char.isupper(): #es no terminal
                    no_terminal_count = True

            if terminal_count >= 1 and not no_terminal_count:
                productive = True
                
        if not productive and k != 'S':
            not_productive.append(k)
    # print(not_productive)

    #=======
    filter_no_productive = {}
    for k,v in rules.items():
        if k not in not_productive:
            new_rules = []
            for production in v:
                productive = True
                for char in production:
                    if char in not_productive:
                        # print(char)
                        productive = False
                if productive:
                    new_rules.append(production)
            if new_rules:
                filter_no_productive[k] = new_rules
    # print(filter_no_productive)

    #====
    rule_s = filter_no_productive['S']
    # print(f'rules s: {rule_s}')
    not_redundants = {}
    inalcanzable = []

    for k, v in filter_no_productive.items():
        alcanzable = False
        for production in v:
            # print(production)
            # print(f'k:{k}')
            for i in rule_s:          
                # print(i)
                # print(f'---{k}')
                if k in i or k == 'S':
                    alcanzable = True
            if not alcanzable:
                inalcanzable.append(k)
    # print(inalcanzable)

    for k,v in filter_no_productive.items():
        if k not in inalcanzable:
            not_redundants[k] = v
    # print(not_redundants)

    return not_redundants

In [105]:
simbolos_redundantes(rules)

{'S': ['aC'], 'C': ['ad']}

## Simplicidad por reglas unitarias

In [82]:
def simplicidad_unitaria(rules):
    unitaria_terminal = {}
    for key in rules:
        unitaria = list(key)
        contador = 0
        while contador < len(unitaria):
            B = unitaria[contador]
            if B in rules:
                for production in rules[B]:
                    if len(production) == 1 and production.isupper() and production not in unitaria:
                        unitaria.append(production)
            contador += 1
        unitaria_terminal[key] = unitaria
    
    print(f'unitaria terminal: {unitaria_terminal}')

    new_rules = {}
    for key in rules:
        new_rules[key] = []
        for i in unitaria_terminal[key]:
            if i in rules:
                for production in rules[i]:
                    if not(len(production) == 1 and production.isupper()):
                        new_rules[key].append(production)

    print(f'new rules: {new_rules}')

    no_unitarias = ['S']
    stack = ['S']

    while stack:
        current_value = stack.pop()
        if current_value in new_rules:
            for production in new_rules[current_value]:
                for char in production:
                    if char.isupper() and char not in no_unitarias:
                        no_unitarias.append(char)
                        stack.append(char)
    print(f'stack:{stack}')
    print(f'no unitary: {no_unitarias}')

    final = {}

    for key in no_unitarias:
        if key in new_rules:
            final[key] = new_rules[key]

    return final

In [84]:
r = {
    'S': ['bA', 'aB'],
    'A': ['bAA', 'aS', 'a'],
    'B': ['aBB', 'bS', 'a']
}
simplicidad_unitaria(rules)

unitaria terminal: {'S': ['S'], 'A': ['A', 'E'], 'B': ['B', 'C', 'D'], 'C': ['C', 'D'], 'D': ['D'], 'E': ['E']}
new rules: {'S': ['AB'], 'A': ['a'], 'B': ['b'], 'C': ['b'], 'D': ['b'], 'E': ['a']}
stack:[]
no unitary: ['S', 'A', 'B']


{'S': ['AB'], 'A': ['a'], 'B': ['b']}

## Simplicidad por epsilon-producciones

In [440]:
def epsilon_producciones(rules):
    epsilon = [] #todos los elementos que pueden llevar a epsilon

    for k, v in rules.items():
        if 'epsilon' in v and k not in epsilon: #una de las producciones contiene epsilon?
            epsilon.append(k)
    # print(epsilon)
    updates = True

    while updates: #mientras exista un epsilon...
        updates = False

        for k, v in rules.items():
            if k in epsilon: #si key está, avanzamos
                continue
            for production in v: #comparar producción contra todas
                condition_epsilon = True #puede haber epsilon

                for char in production: #por cada caracter de producción
                    # print(f'char: {char}')
                    if char not in epsilon: #si caracter no está en epsilon; a no en [X]
                        condition_epsilon = False #no existe epsilon
                        break

                if condition_epsilon: #si hay.. se añade a epsilon
                    # print('yes')
                    epsilon.append(k)
                    updates = True
                    break

    new_rules_epsilon = {}

    for k,v in rules.items():
        new_rules_epsilon[k] = []
        for production in v:
            if production == 'epsilon': #eliminar epsilon
                # print('oh',production)
                continue

            if production not in new_rules_epsilon[k]:
                new_rules_epsilon[k].append(production)
            # print(new_rules_epsilon)

            pos_epsilon = []

            for i in range(len(production)):
                if production[i] in epsilon:
                    pos_epsilon.append(i)

            combinations = [[]]

            for pos in pos_epsilon:
                new_combinations = []
                for c in combinations:
                    new_combinations.append(c)
                    new_combinations.append(c + [pos])

                combinations = new_combinations

            for comb in combinations:
                if not comb:
                    continue
                new_production = ""

                for i in range(len(production)):
                    if i not in comb:
                        new_production += production[i]
                if new_production != "" and new_production not in new_rules_epsilon[k]:
                    new_rules_epsilon[k].append(new_production)
                    
    # print(new_rules_epsilon)
    return new_rules_epsilon

In [443]:
rules = {
    "S": ["aX", "bX"],
    "X": ["a", "b", "epsilon"]  # 'e' representa épsilon
}
epsilon_producciones(rules)

{'S': ['aX', 'a', 'bX', 'b'], 'X': ['a', 'b']}

## Forma Normal de Chomsky (FNC)

In [ ]:
unificacion_no_recursiva(rules)
simbolos_redundantes(rules)
simplicidad_unitaria(rules)
epsilon_producciones(rules)

In [468]:
rules

{'S': ['bA', 'aB'], 'A': ['bAA', 'aS', 'a'], 'B': ['aBB', 'bS', 'a']}

In [469]:
# #1.Revisar si tiene recursividad a la izquierda: inmediata
# for k, v in rules.items():
#     for production in v:
#         if production.startswith(k):
#             print('yes')

#reucrisivdad izquierda indirecta

# rules = {
#     'S': ['Aa', 'b'],
#     'A': ['Sc', 'd']
# }

orden = []
for k, v in rules.items():
    orden.append(k)

for i in range(len(orden)):
    Ai = orden[i]
    for j in range(i):
        Aj = orden[j]
        new = []
        for production in rules[Ai]:
            if production.startswith(Aj):
                for prod_aj in rules[Aj]:
                    new.append(prod_aj + production[1:])
            else:
                new.append(production)
        rules[Ai] = new

#recursion inmediata
new_rules = {}
for i in orden:
    alphas = []
    betas = []
    for production in rules[i]:
        if production.startswith(i):
            alphas.append(production[len(i):])
        else:
            betas.append(production)

    if alphas:
        dash = i + "'"
        new_rules[i] = [beta + dash for beta in betas]
        new_rules[dash] = [alpha + dash for alpha in alphas]
        new_rules[dash].append('epsilon')
    else:
        new_rules[i] = rules[i]


In [470]:
new_rules

{'S': ['bA', 'aB'], 'A': ['bAA', 'aS', 'a'], 'B': ['aBB', 'bS', 'a']}

In [475]:
epsilon_producciones(new_rules)

{'S': ['bA', 'aB'], 'A': ['bAA', 'aS', 'a'], 'B': ['aBB', 'bS', 'a']}

In [476]:
simplicidad_unitaria(new_rules)

{'S': ['bA', 'aB'], 'A': ['bAA'], 'B': ['aBB']}

In [477]:
simbolos_redundantes(new_rules)

{'S': ['bA', 'aB'], 'A': ['bAA', 'aS', 'a']}